In [85]:
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
import numpy as np
import yfinance as yf
s = yf.Ticker("AAPL")
opt = s.option_chain('2020-07-16')
calls = opt.calls
calls['Fair Price'] = (calls['ask']+calls['bid'])/2
puts = opt.puts
puts['Fair Price'] = (puts['ask']+puts['bid'])/2
print(opt)

Options(calls=         contractSymbol       lastTradeDate  strike  lastPrice     bid  \
0   AAPL200717C00115000 2020-07-09 19:02:06 115.000    267.700 265.850   
1   AAPL200717C00120000 2020-06-22 22:37:51 120.000    183.270 270.900   
2   AAPL200717C00135000 2020-07-13 19:35:01 135.000    249.650 246.350   
3   AAPL200717C00140000 2020-07-10 17:06:03 140.000    240.970 240.750   
4   AAPL200717C00145000 2020-07-08 19:06:15 145.000    234.800 235.750   
..                  ...                 ...     ...        ...     ...   
93  AAPL200717C00520000 2020-07-13 18:58:26 520.000      0.020   0.010   
94  AAPL200717C00525000 2020-07-13 17:59:09 525.000      0.020   0.010   
95  AAPL200717C00530000 2020-07-13 18:58:42 530.000      0.020   0.010   
96  AAPL200717C00535000 2020-07-13 19:35:52 535.000      0.010   0.010   
97  AAPL200717C00540000 2020-07-13 19:59:49 540.000      0.020   0.010   

       ask  change  percentChange  volume  openInterest  impliedVolatility  \
0  268.650   0.000 

In [46]:
def Call_Credit_Spread():
    c = calls[calls['volume']>=250]
    l = c[['strike', 'Fair Price']]
    m = l.set_index('strike').to_dict()
    n = m['Fair Price'] 
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    for short in n:
        for long in n:
            if long<=short:
                continue
            else:
                strikeShort = short
                strikeLong = long
                #Selling the cheapest one and then buying the more expensive one, one by one
                premiumShort = n[short]
                premiumLong = n[long]
                #figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                MaxReward = float('{:,.2f}'.format(premiumShort - premiumLong))
                #print("MaxReward = ", MaxReward)
                MaxRisk = float('{:,.2f}'.format(strikeLong - strikeShort - MaxReward))
                #print("MaxRisk = ", MaxRisk)
                if MaxReward == 0:
                    continue
                elif MaxReward>0:
                    #We check this to ignore the cases where the oremium is higher for a higher strikeprice in the money
                    Ratio =  float('{:,.2f}'.format(MaxRisk/MaxReward))
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        best_ratio['Long(Buy)']=long
                        best_ratio['Short(Sell)']=short
                        best_ratio['Risk/Reward Ratio']=Ratio
                        best_ratio['Short Premium'] = n[short]
                        best_ratio['Long Premium'] = n[long]
                        best_ratio['maxRisk'] = MaxRisk
                        best_ratio['maxReward'] = MaxReward
                        
                        
                #print("Ratio = ", Ratio)
        return best_ratio
        



In [47]:
Call_Credit_Spread()


{'Short(Sell)': 250.0,
 'Long(Buy)': 270.0,
 'Risk/Reward Ratio': 0.0,
 'Short Premium': 132.14999999999998,
 'Long Premium': 112.15,
 'maxRisk': 0.0,
 'maxReward': 20.0}

In [80]:
def Put_Credit_Spread():
    p = puts[puts['volume']>=250]
    l = p[['strike', 'Fair Price']]
    m = l.set_index('strike').to_dict()
    n = m['Fair Price']
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    for long in n:
        for short in n:
            if long>=short:
                continue
            else:
                strikeShort = short
                strikeLong = long
                #Selling the cheapest one and then buying the more expensive one, one by one
                premiumShort = n[short]
                premiumLong = n[long]
                #figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                MaxReward = float('{:,.2f}'.format(premiumShort - premiumLong))
                #print("MaxReward = ", MaxReward)
                MaxRisk = float('{:,.2f}'.format(strikeLong - strikeShort - MaxReward))
                MaxRisk*=-1
                #print("MaxRisk = ", MaxRisk)
                if MaxReward == 0:
                    continue
                elif MaxReward>0:
                    #We check this to ignore the cases where the oremium is higher for a higher strikeprice in the money
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        best_ratio['Long(Buy)']=long
                        best_ratio['Short(Sell)']=short
                        best_ratio['Risk/Reward Ratio']=Ratio
                        best_ratio['Short Premium'] = n[short]
                        best_ratio['Long Premium'] = n[long]
                        best_ratio['maxRisk'] = MaxRisk
                        best_ratio['maxReward'] = MaxReward
                        
                        
                #print("Ratio = ", Ratio)
        return best_ratio

In [81]:
Put_Credit_Spread()

{'Short(Sell)': 420.0,
 'Long(Buy)': 235.0,
 'Risk/Reward Ratio': 5.821475110763618,
 'Short Premium': 38.375,
 'Long Premium': 0.005,
 'maxRisk': 223.37,
 'maxReward': 38.37}

In [115]:
def Put_Debit_Spread():
    p = puts[puts['volume']>=250]
    l = p[['strike', 'Fair Price']]
    m = l.set_index('strike').to_dict()
    n = m['Fair Price']

#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    for long in n:
        print("\n")
        for short in n:
            if long<=short:
                continue
            else:
                strikeShort = short
                strikeLong = long
                #Selling the cheapest one and then buying the more expensive one, one by one
                premiumShort = n[short]
                premiumLong = n[long]
                #figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                MaxReward = float('{:,.2f}'.format(strikeLong - strikeShort - premiumLong - premiumShort))
#                 print("MaxReward = ", MaxReward)
                MaxRisk = float('{:,.2f}'.format(premiumLong - premiumShort))
#                 MaxRisk*=-1
#                 print("MaxRisk = ", MaxRisk)
                if MaxReward == 0:
                    continue
                elif MaxReward>0:
                    #We check this to ignore the cases where the oremium is higher for a higher strikeprice in the money
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        best_ratio['Long(Buy)']=long
                        best_ratio['Short(Sell)']=short
                        best_ratio['Risk/Reward Ratio']=Ratio
                        best_ratio['Short Premium'] = n[short]
                        best_ratio['Long Premium'] = n[long]
                        best_ratio['maxRisk'] = MaxRisk
                        best_ratio['maxReward'] = MaxReward
                        
                        
                #print("Ratio = ", Ratio)
        print(best_ratio)

In [116]:
Put_Debit_Spread()



{'Short(Sell)': 0, 'Long(Buy)': 0, 'Risk/Reward Ratio': 2147483647, 'Short Premium': 0, 'Long Premium': 0, 'maxRisk': 0, 'maxReward': 0}


{'Short(Sell)': 235.0, 'Long(Buy)': 255.0, 'Risk/Reward Ratio': 0.0005005005005005005, 'Short Premium': 0.005, 'Long Premium': 0.01, 'maxRisk': 0.01, 'maxReward': 19.98}


{'Short(Sell)': 255.0, 'Long(Buy)': 265.0, 'Risk/Reward Ratio': 0.0, 'Short Premium': 0.01, 'Long Premium': 0.015, 'maxRisk': 0.0, 'maxReward': 9.97}


{'Short(Sell)': 255.0, 'Long(Buy)': 265.0, 'Risk/Reward Ratio': 0.0, 'Short Premium': 0.01, 'Long Premium': 0.015, 'maxRisk': 0.0, 'maxReward': 9.97}


{'Short(Sell)': 255.0, 'Long(Buy)': 265.0, 'Risk/Reward Ratio': 0.0, 'Short Premium': 0.01, 'Long Premium': 0.015, 'maxRisk': 0.0, 'maxReward': 9.97}


{'Short(Sell)': 255.0, 'Long(Buy)': 265.0, 'Risk/Reward Ratio': 0.0, 'Short Premium': 0.01, 'Long Premium': 0.015, 'maxRisk': 0.0, 'maxReward': 9.97}


{'Short(Sell)': 255.0, 'Long(Buy)': 265.0, 'Risk/Reward Ratio': 0.0, 'Short Pre

In [117]:
    c = calls[calls['volume']>=250]
    l = c[['strike', 'Fair Price']]
    m = l.set_index('strike').to_dict()
    n = m['Fair Price'] 
    print(n)

{250.0: 132.14999999999998, 270.0: 112.15, 275.0: 107.17500000000001, 315.0: 67.375, 320.0: 62.3, 330.0: 52.375, 340.0: 42.599999999999994, 345.0: 37.6, 350.0: 32.75, 355.0: 27.95, 360.0: 23.175, 362.5: 20.925, 365.0: 18.725, 367.5: 16.575000000000003, 370.0: 14.875, 372.5: 12.9, 375.0: 11.225, 377.5: 9.600000000000001, 380.0: 7.9750000000000005, 382.5: 6.975, 385.0: 6.025, 387.5: 4.775, 390.0: 3.9499999999999997, 392.5: 3.55, 395.0: 2.71, 397.5: 2.1799999999999997, 400.0: 1.825, 405.0: 1.315, 410.0: 0.9, 415.0: 0.71, 420.0: 0.645, 425.0: 0.45, 430.0: 0.29500000000000004, 435.0: 0.27, 440.0: 0.30000000000000004, 445.0: 0.275, 450.0: 0.14500000000000002, 455.0: 0.13, 460.0: 0.115, 465.0: 0.14, 470.0: 0.065, 475.0: 0.1, 480.0: 0.12000000000000001, 485.0: 0.03, 490.0: 0.04, 495.0: 0.035, 500.0: 0.03, 505.0: 0.03, 510.0: 0.03, 520.0: 0.02, 525.0: 0.02, 530.0: 0.02, 535.0: 0.02, 540.0: 0.015}


In [128]:
def call_Debit_Spread():

#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    for long in n:
        print("\n")
        for short in n:
            if long>=short:
                continue
            else:
                strikeShort = short
                strikeLong = long
                #Selling the cheapest one and then buying the more expensive one, one by one
                premiumShort = n[short]
                premiumLong = n[long]
                #figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                MaxReward = float('{:,.2f}'.format(strikeLong - strikeShort - premiumLong - premiumShort))
                MaxReward*=-1
#                 print("MaxReward = ", MaxReward)
                MaxRisk = float('{:,.2f}'.format(premiumLong - premiumShort))
#                 MaxRisk*=-1
#                 print("MaxRisk = ", MaxRisk)
                if MaxReward == 0:
                    continue
                elif MaxReward>0:
                    #We check this to ignore the cases where the oremium is higher for a higher strikeprice in the money
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        best_ratio['Long(Buy)']=long
                        best_ratio['Short(Sell)']=short
                        best_ratio['Risk/Reward Ratio']=Ratio
                        best_ratio['Short Premium'] = n[short]
                        best_ratio['Long Premium'] = n[long]
                        best_ratio['maxRisk'] = MaxRisk
                        best_ratio['maxReward'] = MaxReward
                        
                        
                #print("Ratio = ", Ratio)
            print(best_ratio)

In [129]:
call_Debit_Spread()



{'Short(Sell)': 270.0, 'Long(Buy)': 250.0, 'Risk/Reward Ratio': 0.07567158531971245, 'Short Premium': 112.15, 'Long Premium': 132.14999999999998, 'maxRisk': 20.0, 'maxReward': 264.3}
{'Short(Sell)': 270.0, 'Long(Buy)': 250.0, 'Risk/Reward Ratio': 0.07567158531971245, 'Short Premium': 112.15, 'Long Premium': 132.14999999999998, 'maxRisk': 20.0, 'maxReward': 264.3}
{'Short(Sell)': 270.0, 'Long(Buy)': 250.0, 'Risk/Reward Ratio': 0.07567158531971245, 'Short Premium': 112.15, 'Long Premium': 132.14999999999998, 'maxRisk': 20.0, 'maxReward': 264.3}
{'Short(Sell)': 270.0, 'Long(Buy)': 250.0, 'Risk/Reward Ratio': 0.07567158531971245, 'Short Premium': 112.15, 'Long Premium': 132.14999999999998, 'maxRisk': 20.0, 'maxReward': 264.3}
{'Short(Sell)': 270.0, 'Long(Buy)': 250.0, 'Risk/Reward Ratio': 0.07567158531971245, 'Short Premium': 112.15, 'Long Premium': 132.14999999999998, 'maxRisk': 20.0, 'maxReward': 264.3}
{'Short(Sell)': 270.0, 'Long(Buy)': 250.0, 'Risk/Reward Ratio': 0.07567158531971245